# Importações

In [1]:
# Importação das bibliotecas
import pandas as pd
import matplotlib

In [2]:
# Importação do banco de dados CSV
dados = pd.read_csv(r'C:\Users\Usuário\Documents\GitHub\Analise_Dados\DadosVacinaAM.csv', sep=";")

C:\Users\Usuário\AppData\Local\Temp\ipykernel_3676\3544363823.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv(r'C:\Users\Usuário\Documents\GitHub\Analise_Dados\DadosVacinaAM.csv', sep=";")


In [3]:
# Listagem das colunas para consulta
dados.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708357 entries, 0 to 1708356
Data columns (total 32 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   document_id                               object 
 1   paciente_id                               object 
 2   paciente_idade                            float64
 3   paciente_dataNascimento                   object 
 4   paciente_enumSexoBiologico                object 
 5   paciente_racaCor_codigo                   float64
 6   paciente_racaCor_valor                    object 
 7   paciente_endereco_coIbgeMunicipio         object 
 8   paciente_endereco_coPais                  object 
 9   paciente_endereco_nmMunicipio             object 
 10  paciente_endereco_nmPais                  object 
 11  paciente_endereco_uf                      object 
 12  paciente_endereco_cep                     object 
 13  paciente_nacionalidade_enumNacionalidade  object 
 14  es

In [4]:
dados.vacina_fabricante_nome.value_counts()

PFIZER                                       675743
ASTRAZENECA/FIOCRUZ                          599125
SINOVAC/BUTANTAN                             330186
JANSSEN                                       42765
Pendente Identificação                        19858
PFIZER - PEDIÁTRICA                           15539
PFIZER - PEDI?TRICA                           12134
ASTRAZENECA                                    4920
Pendente Identifica??o                         3965
PFIZER - PEDIÁTRICA MENOR DE 5 ANOS            2524
SINOVAC                                        1593
PFIZER MANUFACTURING BELGIUM NV - BELGICA         4
FUNDACAO OSWALDO CRUZ                             1
Name: vacina_fabricante_nome, dtype: int64

# Tratamento dos dados

In [5]:
# Seleção das colunas a serem analisadas neste projeto
dados = dados.loc[:, ['paciente_idade','paciente_enumSexoBiologico','paciente_racaCor_valor','paciente_endereco_nmMunicipio','vacina_categoria_nome','vacina_fabricante_nome','vacina_dataAplicacao','vacina_descricao_dose']]

# Filtro a partir da 1ª dose
dados = dados[dados['vacina_descricao_dose'] == '1ª Dose']

# Retirando informações duplicadas
dados.drop_duplicates(inplace=True)

# Retirada de pacientes com erro de digitação na idade
ttt_idade = dados[dados['paciente_idade'] != 221.0]

# Renomeia a coluna com caractere especial
dados = dados.rename(columns={'PFIZER - PEDI?TRICA': 'PFIZER - PEDIATRICA'})

# Criando DataFrame vazio com as colunas desejadas
ttt_col_pfizer = pd.DataFrame(columns=['PFIZER - PEDIÁTRICA', 'PFIZER - PEDIATRICA', 'PFIZER - PEDIÁTRICA MENOR DE 5 ANOS'])

# Juntando as colunas Pfizer em uma única coluna
dados['PFIZER PEDIATRICA'] = pd.concat([ttt_col_pfizer['PFIZER - PEDIÁTRICA'], ttt_col_pfizer['PFIZER - PEDIATRICA'], ttt_col_pfizer['PFIZER - PEDIÁTRICA MENOR DE 5 ANOS']])

# Removendo as colunas originais
dados.drop(['PFIZER - PEDIÁTRICA', 'PFIZER - PEDIATRICA', 'PFIZER - PEDIÁTRICA MENOR DE 5 ANOS'], axis=1, inplace=True)

# Exibe o DataFrame resultante
dados


KeyError: "['PFIZER - PEDIÁTRICA', 'PFIZER - PEDIATRICA', 'PFIZER - PEDIÁTRICA MENOR DE 5 ANOS'] not found in axis"

In [ ]:
# Juntar nomes iguais de coluna vacina_fabricante_nome 
dados.vacina_fabricante_nome.value_counts()

## Quem são as pessoas vacinadas com a 1ª dose (COVID-19) no estado do Amazonas?

In [ ]:
# Quantidade de aplicações de 1ª dose da vacina por Idade, em porcentagem
idade = dados.paciente_idade.value_counts()
tt_idade = idade.sum()
porcent_idade = (idade/tt_idade)*100
porcent_idade.plot(kind='bar')

In [ ]:
# Quantidade de aplicações de 1ª dose da vacina por Raça, em porcentagem
raça = dados.paciente_racaCor_valor.value_counts(normalize=True)
tt_raça = raça.sum()
porcent_raça = (raça/tt_raça)*100
porcent_raça.plot(kind='bar', title='Vacinas 1ª dose por Raça, em porcentagem', ylabel='')

In [ ]:
# Quantidade de aplicações de 1ª dose da vacina por sexo, em porcentagem
sexo = dados.paciente_enumSexoBiologico.value_counts(normalize=True)
sexo

In [ ]:
# Quantidade de aplicações de 1ª dose da vacina por Município, em porcentagem
tt_munic = dados.paciente_endereco_nmMunicipio.value_counts()
tt_pac = tt_munic.sum()
porcent_munic = (tt_munic/tt_pac)*100
porcent_munic

## Problema
Há diferença entre o perfil das pessoas que tomaram a segunda dose no Amazonas?
Qual a distância média entre a casa e o local de aplicação?
 (contabilizar 1 dose e 2 doses e 3 doses)

In [ ]:
# Quantidade total de aplicação da 1ª dose da vacina por população de risco, em porcentagem
pop_risco = dados.vacina_categoria_nome.value_counts()
tt_cat = pop_risco.sum()
porcent_pop_risco = (pop_risco/tt_cat)*100
porcent_pop_risco = round(porcent_pop_risco, 2)
porcent_pop_risco

In [ ]:
# Fabricante de vacina por população de risco (trabalhadores da saúde)
pop_risco = dados[(dados['vacina_categoria_nome'] != 'Faixa Etária')]
grupo_pop_risco = pop_risco.groupby('vacina_categoria_nome')['vacina_fabricante_nome'].value_counts()
grupo_trab = grupo_pop_risco['Trabalhadores de Saúde']
tt_trab = grupo_pop_risco['Trabalhadores de Saúde'].sum()
porcent_trab = (grupo_trab/tt_trab)*100
porcent_trab = round(porcent_trab, 2)
porcent_trab

In [ ]:
# Fabricante de vacina por população de risco (comorbidades)
pop_risco = dados[(dados['vacina_categoria_nome'] != 'Faixa Etária')]
grupo_pop_risco = pop_risco.groupby('vacina_categoria_nome')['vacina_fabricante_nome'].value_counts()
grupo_comorb = grupo_pop_risco['Comorbidades']
tt_comorb = grupo_pop_risco['Comorbidades'].sum()
porcent_comorb = (grupo_comorb/tt_comorb)*100
porcent_comorb = round(porcent_comorb, 2)
porcent_comorb

In [ ]:
# Verificar primeira data de aplicação de vacina e apresentar as populações vacinadas
dados['vacina_dataAplicacao'] = pd.to_datetime(dados['vacina_dataAplicacao'], format='%Y-%m-%d')
dados_ord = dados.groupby(['vacina_dataAplicacao', 'vacina_categoria_nome']).size().reset_index(name='count')
dados_ord = dados_ord.sort_values(by='vacina_dataAplicacao')

In [ ]:
#